In [4]:
import pandas as pd
import numpy as np

import zlib
import datetime

import asyncio
import aiohttp
import requests as r

import time
from functools import wraps

from sklearn.ensemble import (
    GradientBoostingClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [98]:
# Модели, доступные для обучения

model_mapper = {
    'GradientBoostingClassifier': GradientBoostingClassifier,
    'RandomForestClassifier': RandomForestClassifier,
    'LogisticRegression': LogisticRegression,
    'SVM': SVC,
    'KNeighborsClassifier': KNeighborsClassifier,
}

## Подготовка данных

In [99]:
# Генерация данных и приведение их к спискам

X = np.random.randint(0, 100, size=(1000, 150))
y = np.random.randint(0, 100, size=(1000, 1))

X = X.tolist()
y = y.ravel().tolist()

In [100]:
# Декоратор для замера времени выполнения функции

def time_of_function(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        start_time = datetime.datetime.now()
        res = function(*args, **kwargs)
        print(datetime.datetime.now() - start_time)
        return res
    return wrapped

## Синхронное выполнение

In [101]:
@time_of_function
def sync_fitting(model, model_params=None, num_of_iterations=1):
    """
    Функция для синхронного обучения моделей
    """

    model = model(**model_params) if model_params else model() 
    
    for _ in range(num_of_iterations):
        model.fit(X, y)

Посчитаем время обучения моделей синхронно

In [102]:
sync_fitting(LogisticRegression, num_of_iterations=1)

0:00:00.877677


/Users/nikolskijvladimir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [103]:
sync_fitting(LogisticRegression, num_of_iterations=2)

/Users/nikolskijvladimir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0:00:01.547888


/Users/nikolskijvladimir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [104]:
sync_fitting(LogisticRegression, num_of_iterations=4)

/Users/nikolskijvladimir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nikolskijvladimir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

0:00:03.350181


/Users/nikolskijvladimir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Взаимодействие с серверной частью

In [105]:
URL = 'http://0.0.0.0:8000'

Запустим сервер в фоновом режиме

In [106]:
!make run_daemon

docker-compose -f docker-compose.yml down && \
	docker-compose -f docker-compose.yml build && \
	docker-compose -f docker-compose.yml up -d app
[+] Running 0/0
 ⠋ Container ml_server-app-1  Stopping                                     0.1s
[+] Running 0/1
 ⠙ Container ml_server-app-1  Stopping                                     0.2s
[+] Running 0/1
 ⠹ Container ml_server-app-1  Stopping                                     0.3s
[+] Running 0/1
 ⠸ Container ml_server-app-1  Stopping                                     0.4s
[+] Running 0/1
 ⠼ Container ml_server-app-1  Stopping                                     0.5s
[+] Running 0/1
 ⠴ Container ml_server-app-1  Stopping                                     0.6s
[+] Running 0/1
 ⠦ Container ml_server-app-1  Stopping                                     0.7s
[+] Running 0/1
 ⠧ Container ml_server-app-1  Stopping                                     0.8s
[+] Running 0/1
 ⠇ Container ml_server-app-1  Stopping                                  

[+] Building 1.6s (11/11) FINISHED                                              
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 69B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3-slim-buster    1.5s
 => [1/6] FROM docker.io/library/python:3-slim-buster@sha256:1d870a66cb77  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 927B                                          0.0s
 => CACHED [2/6] WORKDIR /app                                              0.0s
 => CACHED [3/6] RUN mkdir data                                            0.0s
 => CACHED [4/6] COPY ./requirements.txt /app/requirements.txt             0.0s
 => CACHED [5/6] RUN pip install -r req

In [107]:
!docker ps

CONTAINER ID   IMAGE           COMMAND                  CREATED        STATUS                  PORTS                    NAMES
fae20d2953b9   ml_server-app   "uvicorn src.ml_serv…"   1 second ago   Up Less than a second   0.0.0.0:8000->8000/tcp   ml_server-app-1


In [108]:
# Проверим, что сервер запущен и обрабатывает запросы

health_check = r.get(url=f'{URL}/health')
assert health_check.ok

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# Соберем тело запроса для обучения. 

req_1_body = {
    "X": X,
    "y": y,
    "config":{
        "file_name": "log_reg_1",
        "model": "LogisticRegression"
    }
}

req_2_body = {
    "X": X,
    "y": y,
    "config":{
        "file_name": "log_reg_2",
        "model": "LogisticRegression"
    }
}

In [ ]:
import nest_asyncio
nest_asyncio.apply()

async def async_requests():
    async with aiohttp.ClientSession() as session:
        req_1 = await session.post(url=f'{URL}/fit', json=req_1_body, chunked=True)
        req_2 = await session.post(url=f'{URL}/fit', json=req_2_body, chunked=True)
        
        assert req_1.ok
        assert req_2.ok
        
        print('Запросы обработаны успешно')
        


loop = asyncio.get_event_loop()
task = loop.create_task(async_requests())
loop.run_until_complete(task)

In [109]:
resp_1 = r.get(url=f'{URL}/status/log_reg_1')
resp_2 = r.get(url=f'{URL}/status/log_reg_2')

In [110]:
print(resp_1.json()['start'])
print(resp_2.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [74]:
!make stop_all_docker

docker-compose -f docker-compose.yml down
[+] Running 0/0
 ⠋ Container ml_server-app-1  Stopping                                     0.1s
[+] Running 0/1
 ⠙ Container ml_server-app-1  Stopping                                     0.2s
[+] Running 0/1
 ⠹ Container ml_server-app-1  Stopping                                     0.3s
[+] Running 0/1
 ⠸ Container ml_server-app-1  Stopping                                     0.4s
[+] Running 0/1
 ⠼ Container ml_server-app-1  Stopping                                     0.5s
[+] Running 0/1
 ⠴ Container ml_server-app-1  Stopping                                     0.6s
[+] Running 0/1
 ⠦ Container ml_server-app-1  Stopping                                     0.7s
[+] Running 0/1
 ⠧ Container ml_server-app-1  Stopping                                     0.8s
[+] Running 0/1
 ⠇ Container ml_server-app-1  Stopping                                     0.9s
[+] Running 1/1
 ⠿ Container ml_server-app-1  Removed                                      0.9